In [177]:
# Libraries to be used in the model
import pandas as pd
import numpy as py
import datetime as dt

# Plotiing libraries
import matplotlib.pyplot as plt

#Model Libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

from statistics import mean
from sklearn.metrics import mean_absolute_error

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

######################################3

# Get unique values per column
def print_unique_values_per_column(d):
    for c in d.columns:
        if (d[c].nunique() < 30):
            print(c, d[c].nunique(), d[c].dtype, d[c].unique())
        else:
            print(c, d[c].nunique(), d[c].dtype)

# Print missing values
def print_missing_values(d):
    missing_total = d.isnull().sum().sort_values(ascending=False)
    missing_percent = (d.isnull().sum()/d.isnull().count()).sort_values(ascending=False)
    missing = pd.concat([missing_total, missing_percent], axis=1, keys=['Total', 'Percent'])
    missing = missing[missing['Percent'] > 0]
    print(missing)

def get_columns_with_nulls(d):
    missing_total = d.isnull().sum().sort_values(ascending=False)
    missing_percent = (d.isnull().sum()/d.isnull().count()).sort_values(ascending=False)
    missing = pd.concat([missing_total, missing_percent], axis=1, keys=['Total', 'Percent'])
    missing = missing[missing['Percent'] > 0]
    return missing.index.values

# Custom accuracy evaluation
def accuracy_score(estimator, X, y):
    estimator.fit(X, y)
    y_pred = estimator.predict(X)
    accuracy = mean(1 - mean_absolute_error(y, y_pred)/y_pred)
    return accuracy

In [179]:
# Reading the data
test = pd.read_csv('test.csv')
train = pd.read_csv('train_data.csv')
train.head()

,ofd_date,country_code,fc_codes,station_code,OFD,Slam,Earlies_Exp,Earlies_Rec,MNR_SNR_Exp,Rollover,Returns,R_Sideline,Sideline
0,2021-06-30,C,"F6, F8, F14, F17",D33,14594,14568,782,896,615,767,35,2,4
1,2021-06-30,C,"F6, F8, F9, F14, F17, F18",D37,12736,13111,655,823,211,29,17,2,1
2,2021-06-30,C,"F1, F4, F6, F7, F13, F15, F16",D34,14562,15651,1028,1910,225,35,47,3,1
3,2021-06-30,C,"F2, F6, F7, F10, F12, F13, F14, F15, F19",D45,11165,11467,514,769,56,39,29,0,1
4,2021-06-30,C,"F6, F8, F13, F14, F17",D50,10006,10423,399,842,52,60,65,1,1


In [ ]:
#Check the ammount of null data
train.isna().sum()

In [180]:
# Data transformation

## 1) Standarized the numerical data (if needed)
#for column in ['']:
#    train[column] = (train[column] - train[column].mean()) / train[column].std()
#train

## 2) Create the target column
train["Target"] = train['Earlies_Exp'] - train['MNR_SNR_Exp']

## 3) Create the column of days of the week with the following coding
"""
0 = Monday
1 = Tuesday
2 = Wednesday
3 = Thrusday
4 = Friday
5 = Saturday
6 = Sunday
""" 
train["ofd_date"] = pd.to_datetime(train["ofd_date"])
train["dayOdWeek"] = train['ofd_date'].dt.day_of_week
#train["dayOfWeekName"] = train['ofd_date'].dt.day_name()

## 4) Create the colums per FC code.
train = pd.concat([train, train['fc_codes'].str.get_dummies(sep=', ')], axis=1)

# 5) Get the categorical data to be separated
train = pd.get_dummies(train, prefix = ['country_'], columns = ['country_code'])

# 6) Drop no relevant columns
train = train.drop(columns=['ofd_date','fc_codes'])

train.head()


,station_code,OFD,Slam,Earlies_Exp,Earlies_Rec,MNR_SNR_Exp,Rollover,Returns,R_Sideline,Sideline,...,F55,F56,F6,F7,F8,F9,country__A,country__B,country__C,country__D
0,D33,14594,14568,782,896,615,767,35,2,4,...,0,0,1,0,1,0,0,0,1,0
1,D37,12736,13111,655,823,211,29,17,2,1,...,0,0,1,0,1,1,0,0,1,0
2,D34,14562,15651,1028,1910,225,35,47,3,1,...,0,0,1,1,0,0,0,0,1,0
3,D45,11165,11467,514,769,56,39,29,0,1,...,0,0,1,1,0,0,0,0,1,0
4,D50,10006,10423,399,842,52,60,65,1,1,...,0,0,1,0,1,0,0,0,1,0


In [161]:
#Check the correlation of the variables with the target column
corr = train.corr()
corr["Target"].sort_values(ascending=False)

Target         1.000000
Earlies_Exp    0.755037
Earlies_Rec    0.400709
OFD            0.290058
Sideline       0.265911
                 ...   
F42           -0.184277
F43           -0.194223
F48           -0.199170
country__D    -0.231282
MNR_SNR_Exp   -0.552011
Name: Target, Length: 70, dtype: float64

In [163]:
#Preparing the variables for the model.
Y = train['Target']
X = train.drop(['Target'], axis=1)

In [169]:
#Model creation: Linear Regression 
scores = cross_val_score(LogisticRegression(), X, Y, cv=2, scoring='roc_auc')
print(scores)

[nan nan]


C:\Users\Mario Jesus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
C:\Users\Mario Jesus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
2 fits failed out of a total of 2.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Mario Jesus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Use

In [165]:
#Model creation: Logistic Regression



In [166]:
#Model creation: SVC

